

<H1>Методы интеллектуального анализа текстов</H1>
<H2>Тема 3. Коррекция очепяток</H2>




### Задание 3:
Написать функцию генерации опечаток в соответствии с положением знаков на клавиатуре.

Импортируем нужные библиотеки.

In [75]:
import re
import random as rd

Записываем в переменную предложение, где нужно сгенерировать ошибки.

In [76]:
sent = 'А что? Так можно было?'  # sent = input()

Проверяем на каком языке было введено предложение.

In [77]:
reg_en = re.compile(r'[a-zA-Z]')
reg_ru = re.compile(r'[а-яА-я]')

if reg_en.match(sent):
    lang = 'EN'
elif reg_ru.match(sent):
    lang = 'RU'
else:
    raise NameError('No Supported Language Found')

Списки символов с клавиатуры.

In [78]:
keylist_en = [['1', '2', '3', '4', '5', '6', '7', '8', '8', '0', '-', '='],
              ['q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p', '[', ']'],
              ['a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', ';', '\'', '\\'],
              ['z', 'x', 'c', 'v', 'b', 'n', 'm', ',', '.', '/']]

keylist_ru = [['1', '2', '3', '4', '5', '6', '7', '8', '8', '0', '-', '='],
              ['й', 'ц', 'у', 'к', 'е', 'н', 'г', 'ш', 'щ', 'з', 'х', 'ъ'],
              ['ф', 'ы', 'в', 'а', 'п', 'р', 'о', 'л', 'д', 'ж', 'э', '\\'],
              ['я', 'ч', 'с', 'м', 'и', 'т', 'ь', 'б', 'ю', '.']]

Функция генерации опечаток.

In [104]:
def generate_typos(keylist, sentence):
    p_typo = 0.270  # Вероятность появления опечатки
    sentence_w_typos = ''  # Переменная для нового предложения с опечатками

    # Основной цикл перебора символов в исходном тексте
    for sym in sentence:
        # Определяем будет ли опечатка или нет.
        if p_typo < rd.random() or sym == ' ':
            sentence_w_typos += ''.join(sym)
            continue

        # Определяем находится ли символ в нижнем регистре
        if sym.islower():
            low_flag = True
        else:
            low_flag = False

        # Находим символ в списке символов и записываем его позицию
        found_flag = False
        for line in keylist:
            if sym.lower() in line:
                sym_pos = line.index(sym.lower())
                line_pos = keylist.index(line)
                found_flag = True
                break

        # Если символ не найден, то оставляем этот символ
        if not found_flag:
            sentence_w_typos += ''.join(sym)
            continue

        # Пробуем получить значения соседних элементов обрабатываемого символа
        # В случае успеха, записываем эти значения в массив возможных опечаток
        typos_list = []
        for i in range(-1, 1):
            for j in range(-1, 1):
                try:
                    typos_list.append(keylist[line_pos + i][sym_pos + j])
                except IndexError:
                    pass

        # Случайно выбираем, какой символ (из возможных) поставить вместо исходного,
        # учитывая исходный регистр
        p_typo_var = rd.randint(1, len(typos_list) - 1)
        if low_flag:
            sentence_w_typos += ''.join(typos_list[p_typo_var].lower())
        else:
            sentence_w_typos += ''.join(typos_list[p_typo_var].upper())

    return sentence_w_typos

Используем нашу функцию в зависимости от исходного языка.

In [109]:
if lang == 'EN':
    typo_message = generate_typos(keylist_en, sent)
else:
    typo_message = generate_typos(keylist_ru, sent)

In [121]:
print(typo_message)

А чтг? Так сож6о былг?


### Задание 4:
Написать функцию сравнения векторов, составленных из частотностей входящих в слово букв (CountVectorizer). Помимо верных вариантов слов из примеров использовать в небольшом словаре своего метода дополнительно похожие и непохожие на верные слова русского языка.

Импортируем нужные библиотеки.

In [281]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

Функция сравнения векторов.

In [299]:
def vector_compare(w1, w2):
    # Используем CountVectorizer с посимвольным анализатором с диапазоном n-грамм от 1 до 2 символов
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(1, 2))
    terms = vectorizer.fit_transform([w1, w2]) # Получим термины наших слов
    terms_4_cos = terms.toarray() # Преобразуем в массив для удобства последующих манипуляций
    terms = terms_4_cos.tolist() #  Преобразуем в список для удобства последующих манипуляций

    # Добавляем слова в список для более красивого и наглядного вывода
    terms[0].insert(0, w1)
    terms[1].insert(0, w2)

    # Получаем названия фич для преобразований
    features = list(vectorizer.get_feature_names_out())
    features.insert(0, '') # Добавляем пустой блок, чтобы выровнять списки при выводе

    # Объединяем списки в один
    terms.insert(0, features)

    # Создаем датафрейм с нашим списком
    df = pd.DataFrame(terms)
    # Выводим датафрейм без индексов и без заголовков
    print(df.to_string(index=False, header=False))


    first_vector = np.array(terms_4_cos[0]) # Помещаем в массив первый вектор
    second_vector = np.array(terms_4_cos[1]) # Помещаем в массив второй вектор
    result = cosine_similarity(first_vector.reshape(1, -1), second_vector.reshape(1, -1))
    print('Косинусная близость между векторами: ', result[0][0], end='\n')

In [300]:
vector_compare('Перевод', 'Пиревод')
vector_compare('интернете', 'инете')
vector_compare('лучше', 'лутше')
vector_compare('созвон', 'ссозвон')
vector_compare('Опечатка', 'Очепятка')
#vector_compare('частнопредпринимательский', 'частнапридпренемательской')

        в во д е ев ер и ир о од п пе пи р ре
Перевод 1  1 1 2  1  1 0  0 1  1 1  1  0 1  1
Пиревод 1  1 1 1  1  0 1  1 1  1 1  0  1 1  1
Косинусная близость между векторами:  0.7877263614433762
          е ер ет и ин н не нт р рн т те
интернете 3  1  1 1  1 2  1  1 1  1 2  2
    инете 2  0  1 1  1 1  1  0 0  0 1  1
Косинусная близость между векторами:  0.8958280175293669
      е л лу т тш у ут уч ч чш ш ше
лучше 1 1  1 0  0 1  0  1 1  1 1  1
лутше 1 1  1 1  1 1  1  0 0  0 1  1
Косинусная близость между векторами:  0.6666666666666667
        в во з зв н о оз он с со сс
 созвон 1  1 1  1 1 2  1  1 1  1  0
ссозвон 1  1 1  1 1 2  1  1 2  1  1
Косинусная близость между векторами:  0.9417419115948376
         а ат е еп еч к ка о оп оч п пе пя т тк ч ча че я ят
Опечатка 2  1 1  0  1 1  1 1  1  0 1  1  0 1  1 1  1  0 0  0
Очепятка 1  0 1  1  0 1  1 1  0  1 1  0  1 1  1 1  0  1 1  1
Косинусная близость между векторами:  0.6262242910851494
